In [1]:
from pathlib import Path
import shutil # shutil is still useful for high-level file operations like copy2

src_root = Path('data/sim_cases')
dst_root = Path('data/finit_files')

# Create destination directory if it doesn't exist
dst_root.mkdir(parents=True, exist_ok=True) # pathlib's mkdir is very convenient

finit_file_names = []

# Iterate through each simulation case folder
# Using .iterdir() directly on the Path object
for case_path in src_root.iterdir():
    # .is_dir() is a method on Path objects
    if case_path.is_dir():
        # Find the finit file (case-insensitive, endswith .FINIT)
        # Iterate over files within the case_path directory
        for src_file in case_path.iterdir():
            # .is_file() to ensure it's a file, not a subdirectory
            # .suffix gives the extension, .upper() for case-insensitivity
            if src_file.is_file() and src_file.suffix.upper() == '.FINIT':
                # .name gives just the filename (e.g., 'CASE_A.FINIT')
                dst_file = dst_root / src_file.name # Use / operator to join Path objects
                shutil.copy2(src_file, dst_file)
                finit_file_names.append(src_file.name) # Append the file name

# Print the list of finit file names
# print('Copied FINIT files:')
# for fname in finit_file_names:
#     print(fname)
print(f'Total FINIT files copied: {len(finit_file_names)}')

Total FINIT files copied: 2


In [2]:
finit_file_names

['JD_BASECASE_2.FINIT', 'JD_BASECASE_1.FINIT']

In [3]:
from pathlib import Path
from extract_properties_from_finit import extract_properties_from_finit
from generate_full_properties import generate_full_properties
from CMG_format_compress import CMG_format_compress

finit_dir = 'data/finit_files'
save_dir = 'results/properties'

finit_dir = Path(finit_dir)
save_dir = Path(save_dir)
save_dir.mkdir(exist_ok=True)

for finit_file_name in finit_file_names:
    finit_file_path = finit_dir / finit_file_name
    
    # STEP 1: Extract properties from FINIT file for active cells only
    extracted_property_dict = extract_properties_from_finit(
        finit_file_path = finit_file_path,
        keywords = ['PORO', 'PERMX', 'ACTID'],
        is_save = False,
        save_dir = save_dir,
        save_name = finit_file_name.split('.')[0],
        show_summary = False
    )

    # STEP 2: Generate full properties for all cells (filled inactive cells with zeros)
    full_property_dict = generate_full_properties(
        property_dict = extracted_property_dict,
        property_list = ['PORO', 'PERMX'], 
        total_cells = 989001,
        is_save = False,
        save_dir = save_dir,
        save_name = finit_file_name.split('.')[0],
        show_summary = False,
        is_j_reversed = True,
        grid_shape = (107, 117, 79),
        )


    # STEP 3: Compress full properties to CMG format (repeated values as N*value)
    for key in ['PORO', 'PERMX']:
        CMG_format_compress(
            array = full_property_dict[key], 
            keyword = key, 
            max_line_length = 80,
            show_summary = True,
            save_dir = save_dir,
            save_name = finit_file_name.split('.')[0]
        )



COMPRESSED PORO SUMMARY
Compressed 989,001 values into 54969 lines
Compression ratio: 18.0:1
Total cells: 989,001
PORO - Mean: 0.082804
PORO - Min: 0.000000
PORO - Max: 0.399547
Saved compressed PORO data to: results/properties/JD_BASECASE_2_PORO.dat

COMPRESSED PERMX SUMMARY
Compressed 989,001 values into 39237 lines
Compression ratio: 25.2:1
Total cells: 989,001
PERMX - Mean: 87.341081
PERMX - Min: 0.000000
PERMX - Max: 5000.000000
Saved compressed PERMX data to: results/properties/JD_BASECASE_2_PERMX.dat

COMPRESSED PORO SUMMARY
Compressed 989,001 values into 54969 lines
Compression ratio: 18.0:1
Total cells: 989,001
PORO - Mean: 0.082804
PORO - Min: 0.000000
PORO - Max: 0.399547
Saved compressed PORO data to: results/properties/JD_BASECASE_1_PORO.dat

COMPRESSED PERMX SUMMARY
Compressed 989,001 values into 39237 lines
Compression ratio: 25.2:1
Total cells: 989,001
PERMX - Mean: 87.341081
PERMX - Min: 0.000000
PERMX - Max: 5000.000000
Saved compressed PERMX data to: results/propert

In [5]:
import numpy as np
data_test = np.load('results/full_PORO.npy')

print(data_test)

print(data_test.shape)

data_test


[0. 0. 0. ... 0. 0. 0.]
(989001,)


array([0., 0., 0., ..., 0., 0., 0.])

In [6]:
import numpy as np

# Define the shape
shape = (107, 117, 79)

# Load the 1D array
arr_1d = np.load('results/full_PORO.npy')

# Reshape to 3D
arr_3d = arr_1d.reshape(shape)

# Flip the j-axis (axis=1)
arr_3d_flipped = arr_3d[:, ::-1, :]

# Flatten back to 1D (row-major order, which matches the original)
arr_1d_flipped = arr_3d_flipped.flatten()

# Save the rearranged 1D array
np.save('results/full_PORO_j_reversed.npy', arr_1d_flipped)

print("Rearrangement complete. Saved as results/full_PORO_j_reversed.npy")

Rearrangement complete. Saved as results/full_PORO_j_reversed.npy
